In [1]:
from sklearn.preprocessing import LabelEncoder
from bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from keras_preprocessing import sequence
import sklearn
import numpy as np
import re

In [2]:
seq_length = 10
def string_to_array(my_string):
    my_string = my_string.lower()
    my_string = re.sub('[^acgt0]', 'z', my_string)
    my_array = np.array(list(my_string))
    return my_array

def readmaf(start, end, filename):
    count = 0
    oldlist =[]
    align = AlignIO.parse(filename, "maf")
    for multiple_alignment in align:
        count = count+1
        if count >start and count<=end:
            oldlist.append(multiple_alignment)
        elif count>end:
            break
    return oldlist

#oldlist stores MSAs
oldlist = readmaf(0, 100000, "data/chr2.anc.maf")
print(oldlist[1][1])
print(dir(oldlist[1]))
print(oldlist[1][0].id)
#print(oldlist[2])

ID: panTro4.chr2A
Name: panTro4.chr2A
Number of features: 0
/start=4703
/size=1
/strand=1
/srcSize=113622374
Seq('a', SingleLetterAlphabet())
['__add__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_alphabet', '_annotations', '_append', '_get_per_column_annotations', '_per_col_annotations', '_records', '_set_per_column_annotations', '_str_line', 'add_sequence', 'annotations', 'append', 'column_annotations', 'extend', 'format', 'get_alignment_length', 'sort']
hg38.chr2


In [3]:
print(len(oldlist))

100000


In [4]:
#alignList stores alignments between direct ancestor and descendent
def getAlign(inputList, keyword):
    alignList = []
    zeros = ''
    for i in range(len(inputList)):
        # index for the dog
        indexDes = -1
        indexAnc = -1

        for j in range(len(inputList[i])):
            if len(inputList[i][j])>=10 and 'hg38.chr2' in inputList[i][j].id  :
                indexDes = j
            elif len(inputList[i][j])>=10 and inputList[i][j].id == '_HPGPNRMPC': #'_RMPC' :
                indexAnc = j
        if indexDes!=-1 and indexAnc!=-1:
            alignment = MultipleSeqAlignment([
                    SeqRecord(inputList[i][indexDes].seq+zeros, id=inputList[i][indexDes].id),
                    SeqRecord(inputList[i][indexAnc].seq+zeros, id=inputList[i][indexAnc].id),
                    #SeqRecord(zeros+inputList[i][indexDes].seq, id=inputList[i][indexDes].id)
                ]
            )
            alignList.append(alignment)
    return alignList

def zeros(length):
    return "0"*length
    
FinalList = []    
anc = []
des = []
keyword = ["_HP","_RM","_FC","_CS","_BO","_PB","_TO","_VC", "_PP"]
alignList = getAlign(oldlist, keyword)
print(len(alignList))
print(alignList[126])
print(len(alignList[126][1].seq))
print(len(alignList[126][1]))
for i in range (100):
    print('printing new alignment -------------------------------------------\n')
    print(alignList[i])

#print(str(alignList[62][1]))
#print('The scikit-learn version is {}.'.format(sklearn.__version__))

54952
SingleLetterAlphabet() alignment with 2 rows and 44 columns
G-GCTCATGCTGCT-------------CCCA-----TGCTGCAG hg38.chr2
G-GCTCATGCTGCT-------------CCCA-----CGCTGCAG _HPGPNRMPC
44
44
printing new alignment -------------------------------------------

SingleLetterAlphabet() alignment with 2 rows and 18 columns
aaaaaattt---tttt-- hg38.chr2
AAAAAATTT---T----- _HPGPNRMPC
printing new alignment -------------------------------------------

SingleLetterAlphabet() alignment with 2 rows and 625 columns
aaaggaaaactacagccattgtgggttatcagattctagtcttg...aag hg38.chr2
AAAGGAAAACTATAGCCCTTGTGGGTTATCAGATTCTAGTCTTG...AAG _HPGPNRMPC
printing new alignment -------------------------------------------

SingleLetterAlphabet() alignment with 2 rows and 13 columns
GGAGAGTGAAGGT hg38.chr2
GGAGAGTGAAGGT _HPGPNRMPC
printing new alignment -------------------------------------------

SingleLetterAlphabet() alignment with 2 rows and 15 columns
GGGTTAAAACATTC- hg38.chr2
-GGTTAAAACGTTC- _HPGPNRMPC
printing new alignme

In [5]:
label_encoder = LabelEncoder()
label_encoder.fit(np.array(['a','c','g','t', 'z']))
encoder = OneHotEncoder(sparse=False, dtype=int, categories=[range(5)])
def one_hot_encoder(my_array, encoder):
    integer_encoded = label_encoder.transform(my_array)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = encoder.fit_transform(integer_encoded)
    #onehot_encoded = np.delete(onehot_encoded, -1, 1)
    return onehot_encoded, encoder

def alignBoth(alignList):
    zeros = '00000'
    h, w = len(alignList), 2;
    Matrix = [[0 for x in range(w)] for y in range(h)]
    for i in range(len(alignList)):
        for j in range(len(alignList[i])):
            Matrix[i][j] = one_hot_encoder(string_to_array(str(alignList[i][j].seq).upper()), encoder)
    return Matrix

# [Index of alignment][1 for Ancester, 0 for Descendant][Always 0]
one_hot_aligned = alignBoth(alignList)
#one_hot_aligned = one_hot_aligned[:][:][0]
one_hot_aligned = np.array(one_hot_aligned)
#print(one_hot_aligned.shape)
#ignorethis, encoder = one_hot_encoder(string_to_array(str(alignList[1][1].seq)),encoder)


print(type(one_hot_aligned))
print(len(one_hot_aligned[126][1]))
print(one_hot_aligned[126][1][0])
#print(encoder.inverse_transform(one_hot_aligned[126][1]))
print(type(one_hot_aligned[126][1]))
# A = [1 0 0 0], C = [0 1 0 0], G = [0 0 1 0], T = [0 0 0 1]
def decoder(array):
    result = ""
    size = len(array)
    for i in range(size):
        if array[i].tolist() == [0, 0, 0, 0, 0]:
            result=result+"0" 
        elif array[i].tolist() == [1, 0, 0, 0, 0]:
            result=result+"A"
        elif array[i].tolist() == [0, 1, 0, 0, 0]:
            result=result+"C"
        elif array[i].tolist() == [0, 0, 1, 0, 0]:
            result=result+"G"
        elif array[i].tolist() == [0, 0, 0, 1, 0]:
            result=result+"T"
        elif array[i].tolist() == [0, 0, 0, 0, 1]:
            result=result+"-"
    return result

def decoderY(array):
    result = ""    
    if array[i].tolist() == [0, 0, 0, 0, 0]:
            result=result+"0" 
    elif array[i].tolist() == [1, 0, 0, 0, 0]:
        result=result+"A"
    elif array[i].tolist() == [0, 1, 0, 0, 0]:
        result=result+"C"
    elif array[i].tolist() == [0, 0, 1, 0, 0]:
        result=result+"G"
    elif array[i].tolist() == [0, 0, 0, 1, 0]:
        result=result+"T"
    elif array[i].tolist() == [0, 0, 0, 0, 1]:
        result=result+"-"
    return result
    
print(decoder(one_hot_aligned[126][1][0]))
print(len(one_hot_aligned))

<class 'numpy.ndarray'>
2
[[0 0 1 0 0]
 [0 0 0 0 1]
 [0 0 1 0 0]
 [0 1 0 0 0]
 [0 0 0 1 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 0 0 1 0]
 [0 0 1 0 0]
 [0 1 0 0 0]
 [0 0 0 1 0]
 [0 0 1 0 0]
 [0 1 0 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 1 0 0 0]
 [0 0 0 1 0]
 [0 0 1 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 0 1 0 0]]
<class 'numpy.ndarray'>
G-GCTCATGCTGCT-------------CCCA-----CGCTGCAG
54952


In [10]:
def vectorize(char_setx, char_sety, sequence_length):
    dataX = []
    dataY = []
    dataX1 = []
    dataY1 = []
    result=[]
    char_setx, char_sety = mutation_with_gap(char_setx, char_sety)
    #print(type(char_sety[0]))
    for i in range(len(char_setx) - sequence_length):
        
        '''for i in char_sety[i: i + sequence_length-1]:
            y1.append(i)'''
        #x1 = char_setx1[i: i + sequence_length]
        x = char_setx[i:i + sequence_length]
        y = char_sety[i: i + sequence_length]
        y1 = [[0, 0, 0, 0, 0]]
        temp = y[:-1]
        temp = temp.tolist()
        for i in temp:
            y1.append(i)
        y1=np.array(y1)
        dataX.append(x)
        dataY.append(y)
        dataY1.append(y1)

    return dataX, dataY, dataY1

def tokenize(one_hot_aligned, sequence_length):
    X = []
    Y = []
    X1 = []
    Y1 = []
    for i in range(len(one_hot_aligned)):
        dataX, dataY, dataY1 = vectorize(one_hot_aligned[i][1][0], one_hot_aligned[i][0][0],  sequence_length)
        X.extend(dataX)
        Y.extend(dataY)
        #X1.extend(dataX1)
        Y1.extend(dataY1)
    return X, Y, Y1

def truncate(one_hot_aligned):
    X = []
    Y = []
    for i in range(len(one_hot_aligned)):
        X.append(one_hot_aligned[i][1][0])
        Y.append(one_hot_aligned[i][0][0])
    X = sequence.pad_sequences(X,
                                 maxlen=30,
                                 truncating='post',
                                 padding='post',
                                 value=0)
    Y = sequence.pad_sequences(Y,
                                 maxlen=30,
                                 truncating='post',
                                 padding='post',
                                 value=0)
    return X,Y

# function for checking if there are at least one mutation and there are no gaps.
def mutation_with_nogap(a, b):
    a = a.tolist()
    b = b.tolist()
    if a.count([0,0,0,0,0,1])>0 or b.count([0,0,0,0,0,1])>0:
        return False
    else:
        return True
    
def mutation_with_gap(a, b):
    a_new = []
    b_new = []
    for i, j in zip(a,b):
        i = i.tolist()
        j= j.tolist()
        if i == [0,0,0,0,1] and j == [0,0,0,0,1]:
            continue
        else:
            a_new.append(i)
            b_new.append(j)
    return np.array(a_new), np.array(b_new)
        


def diffList(a, b):
    count = 0
    length = len(a)
    for i in range(length):
        if a[i] != b[i]:
            count = count+1
    return count

def deleteGap(one_hot_aligned):
    result = one_hot_aligned
    for i in range(len(one_hot_aligned)):
        if one_hot_aligned[i][1][0] == '-' and one_hot_aligned[i][0][0] == '-':
            result.pop(i)
    return result

# Used for non 0 padding
#one_hot_aligned = deleteGap(one_hot_aligned)
X, Y, Y1=tokenize(one_hot_aligned, 10)

# Used for 0 padding
#X, Y = truncate(one_hot_aligned)
X = np.array(X)
Y = np.array(Y)
#X1 = np.array(X1)
Y1 = np.array(Y1)

print(Y.shape)
print(len(X))
pt1 = 2000000
pt2 = 2035000
pt3 = 2085000
X_train = X[:pt1]
X_val = X[pt1:pt2]
X_test = X[pt2:pt3]
y_train = Y[:pt1]
y_val =  Y[pt1:pt2]
y_test = Y[pt2:pt3]
# X_train1 = X1[:pt1]
# X_val1 = X1[pt1:pt2]
# X_test1 = X1[pt2:pt3]
y_train1 = Y1[:pt1]
y_val1 =  Y1[pt1:pt2]
y_test1 = Y1[pt2:pt3]

# X_train = X[:20000]
# X_val = X[20000:23000]
# X_test = X[23000:26000]
# y_train = Y[:20000]
# y_val =  Y[20000:23000]
# y_test = Y[23000:26000]

np.save('prepData/X_train_gap_hg38_v3_chr2_size{}'.format(seq_length), X_train)
np.save('prepData/X_val_gap_hg38_v3_chr2_size{}'.format(seq_length), X_val)
np.save('prepData/X_test_gap_hg38_v3_chr2_size{}'.format(seq_length), X_test)
np.save('prepData/y_train_gap_hg38_v3_chr2_size{}'.format(seq_length), y_train)
np.save('prepData/y_val_gap_hg38_v3_chr2_size{}'.format(seq_length), y_val)
np.save('prepData/y_test_gap_hg38_v3_chr2_size{}'.format(seq_length), y_test)

np.save('prepData/y_train1_gap_hg38_v3_chr2_size{}'.format(seq_length), y_train1)
np.save('prepData/y_val1_gap_hg38_v3_chr2_size{}'.format(seq_length), y_val1)
np.save('prepData/y_test1_gap_hg38_v3_chr2_size{}'.format(seq_length), y_test1)

# np.save('prepData/X_train11mutOnly_v2', X_train1)
# np.save('prepData/X_val11mutOnly_v2', X_val1)
# np.save('prepData/X_test11mutOnly_v2', X_test1)


# np.save('prepData20/X_train1mutOnly', X_train)
# np.save('prepData20/X_val1mutOnly', X_val)
# np.save('prepData20/X_test1mutOnly', X_test)
# np.save('prepData20/y_train1mutOnly', y_train)
# np.save('prepData20/y_val1mutOnly', y_val)
# np.save('prepData20/y_test1mutOnly', y_test)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(X_train[0])
print(y_train[0])
print(y_train1[0])


(2085250, 10, 5)
2085250
(2000000, 10, 5)
(35000, 10, 5)
(50000, 10, 5)
[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [0 0 0 1 0]
 [0 0 0 1 0]
 [0 0 0 1 0]
 [0 0 0 1 0]]
[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [0 0 0 1 0]
 [0 0 0 1 0]
 [0 0 0 1 0]
 [0 0 0 1 0]]
[[0 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [0 0 0 1 0]
 [0 0 0 1 0]
 [0 0 0 1 0]]


In [11]:
# import difflib
# sm=difflib.SequenceMatcher(None,X_test,X_train)
# sm2=difflib.SequenceMatcher(None,y_test,y_train)
for i in range(100):
    print(i, decoder(X[i]))

0 AAAAAATTTT
1 AAAAATTTT-
2 AAAATTTT--
3 AAAGGAAAAC
4 AAGGAAAACT
5 AGGAAAACTA
6 GGAAAACTAT
7 GAAAACTATA
8 AAAACTATAG
9 AAACTATAGC
10 AACTATAGCC
11 ACTATAGCCC
12 CTATAGCCCT
13 TATAGCCCTT
14 ATAGCCCTTG
15 TAGCCCTTGT
16 AGCCCTTGTG
17 GCCCTTGTGG
18 CCCTTGTGGG
19 CCTTGTGGGT
20 CTTGTGGGTT
21 TTGTGGGTTA
22 TGTGGGTTAT
23 GTGGGTTATC
24 TGGGTTATCA
25 GGGTTATCAG
26 GGTTATCAGA
27 GTTATCAGAT
28 TTATCAGATT
29 TATCAGATTC
30 ATCAGATTCT
31 TCAGATTCTA
32 CAGATTCTAG
33 AGATTCTAGT
34 GATTCTAGTC
35 ATTCTAGTCT
36 TTCTAGTCTT
37 TCTAGTCTTG
38 CTAGTCTTGT
39 TAGTCTTGTT
40 AGTCTTGTTT
41 GTCTTGTTTC
42 TCTTGTTTCT
43 CTTGTTTCTT
44 TTGTTTCTTG
45 TGTTTCTTGT
46 GTTTCTTGTC
47 TTTCTTGTCT
48 TTCTTGTCTT
49 TCTTGTCTTT
50 CTTGTCTTTG
51 TTGTCTTTGG
52 TGTCTTTGGG
53 GTCTTTGGGC
54 TCTTTGGGCT
55 CTTTGGGCTA
56 TTTGGGCTAT
57 TTGGGCTATT
58 TGGGCTATTT
59 GGGCTATTTT
60 GGCTATTTTT
61 GCTATTTTTA
62 CTATTTTTAC
63 TATTTTTACC
64 ATTTTTACCT
65 TTTTTACCTC
66 TTTTACCTCT
67 TTTACCTCTT
68 TTACCTCTTT
69 TACCTCTTTG
70 ACCTCTTTGT
71 CCTCTTTGTA
72

In [12]:
for i in range(100):
    print(i, decoder(Y[i]))

0 AAAAAATTTT
1 AAAAATTTTT
2 AAAATTTTTT
3 AAAGGAAAAC
4 AAGGAAAACT
5 AGGAAAACTA
6 GGAAAACTAC
7 GAAAACTACA
8 AAAACTACAG
9 AAACTACAGC
10 AACTACAGCC
11 ACTACAGCCA
12 CTACAGCCAT
13 TACAGCCATT
14 ACAGCCATTG
15 CAGCCATTGT
16 AGCCATTGTG
17 GCCATTGTGG
18 CCATTGTGGG
19 CATTGTGGGT
20 ATTGTGGGTT
21 TTGTGGGTTA
22 TGTGGGTTAT
23 GTGGGTTATC
24 TGGGTTATCA
25 GGGTTATCAG
26 GGTTATCAGA
27 GTTATCAGAT
28 TTATCAGATT
29 TATCAGATTC
30 ATCAGATTCT
31 TCAGATTCTA
32 CAGATTCTAG
33 AGATTCTAGT
34 GATTCTAGTC
35 ATTCTAGTCT
36 TTCTAGTCTT
37 TCTAGTCTTG
38 CTAGTCTTGT
39 TAGTCTTGTT
40 AGTCTTGTTT
41 GTCTTGTTTC
42 TCTTGTTTCT
43 CTTGTTTCTT
44 TTGTTTCTTG
45 TGTTTCTTGT
46 GTTTCTTGTT
47 TTTCTTGTTT
48 TTCTTGTTTC
49 TCTTGTTTCT
50 CTTGTTTCTG
51 TTGTTTCTGG
52 TGTTTCTGGG
53 GTTTCTGGGC
54 TTTCTGGGCT
55 TTCTGGGCTA
56 TCTGGGCTAT
57 CTGGGCTATT
58 TGGGCTATTT
59 GGGCTATTTT
60 GGCTATTTTT
61 GCTATTTTTA
62 CTATTTTTAC
63 TATTTTTACC
64 ATTTTTACCT
65 TTTTTACCTC
66 TTTTACCTCT
67 TTTACCTCTT
68 TTACCTCTTT
69 TACCTCTTTG
70 ACCTCTTTGT
71 CCTCTTTGTA
72

In [13]:
for i in range(100):
    print(i, decoder(Y1[i]))

0 0AAAAAATTT
1 0AAAAATTTT
2 0AAAATTTTT
3 0AAAGGAAAA
4 0AAGGAAAAC
5 0AGGAAAACT
6 0GGAAAACTA
7 0GAAAACTAC
8 0AAAACTACA
9 0AAACTACAG
10 0AACTACAGC
11 0ACTACAGCC
12 0CTACAGCCA
13 0TACAGCCAT
14 0ACAGCCATT
15 0CAGCCATTG
16 0AGCCATTGT
17 0GCCATTGTG
18 0CCATTGTGG
19 0CATTGTGGG
20 0ATTGTGGGT
21 0TTGTGGGTT
22 0TGTGGGTTA
23 0GTGGGTTAT
24 0TGGGTTATC
25 0GGGTTATCA
26 0GGTTATCAG
27 0GTTATCAGA
28 0TTATCAGAT
29 0TATCAGATT
30 0ATCAGATTC
31 0TCAGATTCT
32 0CAGATTCTA
33 0AGATTCTAG
34 0GATTCTAGT
35 0ATTCTAGTC
36 0TTCTAGTCT
37 0TCTAGTCTT
38 0CTAGTCTTG
39 0TAGTCTTGT
40 0AGTCTTGTT
41 0GTCTTGTTT
42 0TCTTGTTTC
43 0CTTGTTTCT
44 0TTGTTTCTT
45 0TGTTTCTTG
46 0GTTTCTTGT
47 0TTTCTTGTT
48 0TTCTTGTTT
49 0TCTTGTTTC
50 0CTTGTTTCT
51 0TTGTTTCTG
52 0TGTTTCTGG
53 0GTTTCTGGG
54 0TTTCTGGGC
55 0TTCTGGGCT
56 0TCTGGGCTA
57 0CTGGGCTAT
58 0TGGGCTATT
59 0GGGCTATTT
60 0GGCTATTTT
61 0GCTATTTTT
62 0CTATTTTTA
63 0TATTTTTAC
64 0ATTTTTACC
65 0TTTTTACCT
66 0TTTTACCTC
67 0TTTACCTCT
68 0TTACCTCTT
69 0TACCTCTTT
70 0ACCTCTTTG
71 0CCTCTTTGT
72